<a href="https://colab.research.google.com/github/S-HATANO1970/otemae/blob/main/%E6%83%85%E5%A0%B1%E6%B4%BB%E7%94%A82_%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワーク
機械学習モデルとの比較

In [4]:
import numpy as np

class SimpleNeuralNetwork:
    """素朴なニューラルネットワーク実装（3層、ReLU活性化関数）"""
    def __init__(self, input_size:int, hidden_size:int, output_size:int, learning_rate:float=0.01, fixed_initial_weights:bool=False):
        """
        コンストラクタ
        :param input_size: 入力層のニューロン数
        :param hidden_size: 隠れ層のニューロン数
        :param output_size: 出力層のニューロン数
        :param learning_rate: 学習率
        :param fixed_initial_weights: 初期重みを固定するかどうか
        """

        # 重みの初期化
        if(fixed_initial_weights):
            np.random.seed(42)
        self.W1:np.ndarray = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        if(fixed_initial_weights):
            np.random.seed(0)
        self.W2:np.ndarray = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        # バイアスの初期化
        self.b1:np.ndarray = np.zeros((1, hidden_size))
        self.b2:np.ndarray = np.zeros((1, output_size))

        # 学習率
        self.learning_rate: float = learning_rate
        # 中間層の出力と活性化関数の値
        self.z1: np.ndarray
        self.a1: np.ndarray
        self.z2: np.ndarray
        self.a2: np.ndarray
        # 損失の履歴
        # 学習過程での損失を記録するリスト
        self.losses: list = []

    def relu(self, x: np.ndarray) -> np.ndarray:
        """
        ReLU活性化関数
        負の値を0にし、正の値はそのまま返す
        :param x: 入力データ
        :return: ReLU適用後の出力
        """
        return np.maximum(0, x)

    def relu_derivative(self, x: np.ndarray) -> np.ndarray:
        """
        ReLUの微分
        :param x: 入力データ
        :return: ReLUの微分値
        """
        return (x > 0).astype(float)

    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        順伝播
        :param X: 入力データ (サンプル数, 入力次元)
        :return: 出力データ (サンプル数, 出力次元)
        """
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.z2  # 出力層は線形
        return self.a2

    def backward(self, X: np.ndarray, y: np.ndarray, output: np.ndarray) -> None:
        """
        逆伝播
        :param X: 入力データ (サンプル数, 入力次元)
        :param y: 正解データ (サンプル数, 出力次元)
        :param output: ネットワークの出力 (サンプル数, 出力次元)
        """
        m = X.shape[0]

        dz2 = output - y
        dW2 = (1/m) * np.dot(self.a1.T, dz2)
        db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)

        da1 = np.dot(dz2, self.W2.T)
        dz1 = da1 * self.relu_derivative(self.z1)
        dW1 = (1/m) * np.dot(X.T, dz1)
        db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)

        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1

    def fit(self, X: np.ndarray, y: np.ndarray, epochs: int=1000, verbose: bool=False, print_weights_every: int=10) -> None:
        """
        学習
        :param X: 入力データ (サンプル数, 入力次元)
        :param y: 正解データ (サンプル数, 出力次元)
        :param epochs: 学習エポック数
        :param verbose: 学習過程を表示するかどうか
        :param print_weights_every: 学習過程を表示する頻度
        """
        for epoch in range(epochs):
            output = self.forward(X)
            loss = np.mean((output - y)**2)
            self.losses.append(loss)
            self.accuracy = np.mean(np.round(output) == y)

            if epochs > 0:
                self.backward(X, y, output)

            if verbose:
                if epoch < 10:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                if epoch <= 100 and epoch % print_weights_every == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                elif epoch <= 1000 and epoch % (print_weights_every*10) == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                elif epoch % (print_weights_every*50) == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                elif epoch == epochs - 1:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """予測"""
        return self.forward(X)

### サンプルデータと学習の実行
'''
X:湿度・気圧とy:天候についてのデータ
x:湿度・気圧
y:雨 0, 晴れ 1
'''
X = np.array([
    [1, 1],
    [1.5,12],
    [2,6],
    [3,1.5],
    [3.5,10],
    [4.5,3],
    [5.5,4],
    [5.8,1],
    [6.5,7],
    [7,11],
    [7.5,4],
    [8,1],
    [8.5,12]
])

y = np.array([
    [0],
    [1],
    [0],
    [0],
    [1],
    [0],
    [1],
    [1],
    [0],
    [0],
    [1],
    [1],
    [1]
])
'''
# --- 1. シンプルなサンプルデータの準備 ---
# 入力データ X: (サンプル数, input_size)
# 例: ORゲートのような論理ゲートを学習
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# 正解データ y: (サンプル数, output_size)
# 例: 論理和 (OR) の結果
y = np.array([
    [0],
    [1],
    [1],
    [1]
])
'''
# --- 2. ネットワークのパラメータ設定 ---
input_size = X.shape[1]      # 入力データの次元 (2)
hidden_size = 3              # 隠れ層のニューロン数 (任意に設定)
output_size = y.shape[1]     # 出力データの次元 (1)
learning_rate = 0.005        # 学習率
epochs = 20000               # エポック数

# --- 3. モデルの初期化と学習前の重み・バイアスの確認 ---
print("--- モデル初期化直後の重みとバイアス ---")
model = SimpleNeuralNetwork(input_size, hidden_size, output_size, learning_rate, fixed_initial_weights=True)
print(f"W1 (初期):\n{model.W1.round(4)}")
print(f"b1 (初期):\n{model.b1.round(4)}")
print(f"W2 (初期):\n{model.W2.round(4)}")
print(f"b2 (初期):\n{model.b2.round(4)}")

# --- 4. モデルの学習（途中の重み・バイアスも表示） ---
print("\n--- 学習開始 ---")
# verbose=Trueで損失の推移を表示
model.fit(X, y, epochs=epochs, verbose=True)
print("--- 学習終了 ---")

# --- 5. 学習後の重みとバイアスの最終確認 ---
print("\n--- 学習終了後の重みとバイアス ---")
print(f"W1 (学習後):\n{model.W1.round(4)}")
print(f"b1 (学習後):\n{model.b1.round(4)}")
print(f"W2 (学習後):\n{model.W2.round(4)}")
print(f"b2 (学習後):\n{model.b2.round(4)}")

# --- 6. 学習後の予測結果の確認 ---
print("\n--- 学習後の予測結果 ---")
predictions = model.predict(X)
print(f"入力 X:\n{X}")
print(f"正解 y:\n{y}")
print(f"予測結果:\n{predictions.round(0)}")
print(f"精度: {model.accuracy:.2f} (正解率)")

# --- 7. 機械学習との比較 ---
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
y = y.reshape(y.size)
model.fit(X,y)
print("\n--- 回帰式の重みとバイアス ---")
print(f"W:\n{model.coef_.round(4)}")
print(f"b:\n{model.intercept_.round(4)}")
y_pred = np.array(model.predict(X)).reshape(y.size,1)
print(f"--- 回帰式予測結果 ---\n{y_pred}")
print(y_pred == y.reshape(y.size,1))
print(f"精度: {np.mean(y_pred == y.reshape(y.size,1)):.2f} (正解率)")

--- モデル初期化直後の重みとバイアス ---
W1 (初期):
[[ 0.4967 -0.1383  0.6477]
 [ 1.523  -0.2342 -0.2341]]
b1 (初期):
[[0. 0. 0.]]
W2 (初期):
[[1.4403]
 [0.3267]
 [0.7991]]
b2 (初期):
[[0.]]

--- 学習開始 ---
Epoch 0, Loss: 372.6505, Output:[[ 3.24 27.4  14.59  6.71 24.44 11.57 14.81  9.16 22.06 30.7  17.27 11.87
  34.56]]
Epoch 0, Loss: 372.6505, Output:[[ 3.24 27.4  14.59  6.71 24.44 11.57 14.81  9.16 22.06 30.7  17.27 11.87
  34.56]]
Epoch 1, Loss: 0.2228, Output:[[-0.03  1.22  0.51  0.01  0.92  0.08  0.16  0.72  0.47  0.92  0.17  1.17
   0.98]]
Epoch 2, Loss: 0.2186, Output:[[-0.03  1.2   0.5   0.01  0.91  0.08  0.16  0.73  0.47  0.9   0.18  1.18
   0.97]]
Epoch 3, Loss: 0.2149, Output:[[-0.03  1.19  0.5   0.01  0.9   0.08  0.16  0.73  0.46  0.89  0.18  1.19
   0.95]]
Epoch 4, Loss: 0.2115, Output:[[-0.03  1.17  0.49  0.02  0.88  0.07  0.15  0.74  0.45  0.88  0.19  1.2
   0.94]]
Epoch 5, Loss: 0.2085, Output:[[-0.03  1.16  0.48  0.02  0.87  0.07  0.15  0.74  0.44  0.87  0.2   1.21
   0.93]]
Epoch 6, Loss: 0.2